# 1. Introduction  
### Describe PE2  
PE2 courses use programming to address practical issues in engineering. Through this, we aim to master the ability to solve automation, data analysis, and modeling problems addressed in the industrial field.  

### Describe your team members and roles for each members  

![재혁](https://media.discordapp.net/attachments/848924937712435253/848925361429020692/1.jpg)

Park Jae Hyuk  
- Leader of the team
- Provides overall direction for the project
- Modified the details of the coding of the team members.
- Giving feedback on coding to team members  

![태현](https://media.discordapp.net/attachments/848924937712435253/848925364272758845/2.png)

Noh Tae-hyun  
- Shared materials to help with coding  
- Create presentation materials  
- Partially coding for each module 

![혜란](https://media.discordapp.net/attachments/848924937712435253/848925359298445332/4.jpg)

Lee Hye-ran  
- Share usage of specific libraries  
- Partially coding for each module  
- Presenting and creating the overall flow of presentation materials  

---

# 2. Object of project
### Explain object of project
First all, this project is intended to create modules for large amounts of data to understand information about the data. We tried to analyze the modulation performance of the modulator easily. When a client provides data to a developer, the developer creates software for the data received from the client.  

![picture](https://cdn.discordapp.com/attachments/848924937712435253/849220087553982474/KakaoTalk_20210601_183631500.png)
---

# 3. Structure of code
### Explain your code
#### 1) src_directory.py  
- It allows Python to perform several functions imported by os, and determines whether a specified path exists.

In [3]:
import os

def create_folder(directory):
    try:
        if not os.path.exists(directory):
            os.makedirs(directory)
    except OSError:
        print('Error: Creating directory. ' + directory)

#### 2) src_extract.py  
- We imported the necessary libraries to extract the data and defined a function that stores data. 
- We used a method of generating an empty list of data and values and then adding values. For example, 'TestSiteInfo' added a value for the item 'Batch'. 
- We added parts that needed more information from the script.

In [5]:
import pandas
import xml.etree.ElementTree as ET
from dateutil.parser import parse
import os
from . import graph
from . import path
from . import directory


def data_save(route):
    tree = ET.parse(route)
    root = tree.getroot()

    columns = ['Lot', 'Wafer', 'Mask', 'TestSite', 'Name', 'Date', 'Row', 'Column', 'ErrorFlag', 'Error description', 'Analysis Wavelength', 'Rsq of Ref. spectrum (Nth)', 'Max transmission of Ref. spec. (dB)', 'Rsq of IV', 'I at -1V [A]', 'I at 1V[A]']
    #  'Script ID', 'Script Version', 'Script Owner',
    #  스크립트 정보

    data = []
    values = []

    TestSiteInfo = root.find('TestSiteInfo').attrib
    values.append(TestSiteInfo['Batch'])
    values.append(TestSiteInfo['Wafer'])
    values.append(TestSiteInfo['Maskset'])
    values.append(TestSiteInfo['TestSite'])
    Modulator = root.find('.//Modulator')
    values.append(Modulator.attrib['Name'])
    PortCombo = Modulator.find('PortCombo')
    values.append(parse(PortCombo.attrib['DateStamp']))
    values.append(TestSiteInfo['DieRow'])
    values.append(TestSiteInfo['DieColumn'])

    if graph.ref_max_Rsq >= 0.99:
        ErrorFlag = 0
        ErrorDescription = 'No Error'
    else:
        ErrorFlag = 1
        ErrorDescription = 'Ref. spec. Error'

    values.append(ErrorFlag)
    values.append(ErrorDescription)
    values.append(root.find(".//DesignParameter[@Name='Design wavelength']").text)
    values.append(graph.ref_max_Rsq)
    values.append(graph.max_IL)

    voltage = list(map(float, root.find('.//IVMeasurement/Voltage').text.split(',')))
    current = list(map(float, root.find('.//IVMeasurement/Current').text.split(',')))
    index1 = voltage.index(-1)
    index2 = voltage.index(1)
    values.append(graph.IV_max_rsq)
    values.append(current[index1])
    values.append(current[index2])

    # 스크립트 정보 추가 필요

    data.append(values)
    # df = pd.DataFrame(data, columns=columns).set_index('Lot')
    df = pandas.DataFrame(data, columns=columns).set_index("Lot")
    save_route = path.path() + '/result/csv'
    directory.create_folder(save_route)
    if not os.path.exists(save_route + '/data.csv'):
        df.to_csv(save_route + '/data.csv', mode='w')
    else:
        df.to_csv(save_route + '/data.csv', mode='a', header=False)


ImportError: attempted relative import with no known parent package

#### 3) src_graph.py  
- We define polyfit, IV, graph functions and find and fit points where the R^square value is maximum. 
- Then, we visualized the fitted graph. 
- We also added options on whether to show or save graphs.

In [7]:
import xml.etree.ElementTree as ET
from xml.etree.ElementTree import parse
import matplotlib.pyplot as plt
import numpy as np
from numpy import exp
import warnings
from lmfit import Parameters, fit_report, minimize, Model
from . import path
from . import directory


def polyfit(x, y, degree):
    results = {}
    coeffs = np.polyfit(x, y, degree)
    results['polynomial'] = coeffs.tolist()
    p = np.poly1d(coeffs)
    yhat = p(x)
    ybar = np.sum(y)/len(y)
    ssreg = np.sum((yhat-ybar)**2)
    sstot = np.sum((y - ybar)**2)
    results['determination'] = ssreg / sstot
    return results


def IV(x, Is, vt): #IV 함수정의
    return Is * (exp(x / vt) - 1)


def graph(route, save, show):
    with warnings.catch_warnings():
        warnings.simplefilter('ignore', np.RankWarning)
        tree = parse(str(route)) #파싱함
        root = tree.getroot()
        plt.figure(figsize=(16, 10)) 

        # IV Measurement
        plt.subplot(2, 2, 4)
        voltage = list(map(float, root.find('.//IVMeasurement/Voltage').text.split(',')))
        current = list(map(float, root.find('.//IVMeasurement/Current').text.split(',')))
        v = np.array(voltage) 
        c = np.array(current)
        gmodel = Model(IV)
        params = gmodel.make_params(Is=1, vt=0.026)
        result = gmodel.fit(c, params, x=v)

        c2 = c - result.best_fit 
        IV_Rsq = {}
        for i in range(1, 20):
            poly = polyfit(v, c2, i)
            IV_Rsq[i] = poly['determination']
        IV_max_key = max(IV_Rsq, key=lambda key: IV_Rsq[key])
        global IV_max_rsq
        IV_max_rsq = IV_Rsq[IV_max_key]
        polyIV = np.poly1d(np.polyfit(v, c2, IV_max_key))(v)

        plt.plot(v, abs(c), 'ok')
        plt.plot(v, abs(result.best_fit + polyIV), '--', label='Highest order term = {}'.format(IV_max_key))
        plt.legend(loc='best')
        plt.xlabel('Voltage[V]')
        plt.ylabel('Current[A]')
        plt.title('IV-analysis')
        plt.yscale('log')

        # Raw Spectrum
        plt.subplot(2, 2, 1)
        TestSiteInfo = root.find('TestSiteInfo')
        TestSite = TestSiteInfo.attrib['TestSite']
        ModulatorName = ".//*[@Name='{}_ALIGN']//".format(TestSite)
        ref_L = list(map(float, root.findtext(str(str(ModulatorName + 'L'))).split(',')))
        ref_IL = list(map(float, root.findtext(str(str(ModulatorName + 'IL'))).split(',')))
        global max_IL
        max_IL = max(ref_IL)
        for wavelengthsweep in root.iter('WavelengthSweep'):
            L = list(map(float, wavelengthsweep.findtext('L').split(',')))
            IL = list(map(float, wavelengthsweep.findtext('IL').split(',')))
            if IL == ref_IL:
                name = 'Reference'
            else:
                name = 'DCBias=' + str(wavelengthsweep.attrib['DCBias']) + 'V'
            plt.plot(L, IL, 'o', label=name)

        plt.legend(loc='lower right')
        plt.xlabel('Wavelength[nm]')
        plt.ylabel('Measured transmissions[dB]')
        plt.title('Transmission spectra - as measured')

        # Fitting
        plt.subplot(2, 2, 3)
        x = np.array(ref_L)
        y = np.array(ref_IL)

        plt.scatter(x, y, facecolor='none', edgecolor='r', alpha=0.06, label='Reference')
        ref_Rsq = {}
        for i in range(4, 7):
            poly = polyfit(x, y, i)
            ref_Rsq[i] = poly['determination']
            fit = np.poly1d(poly['polynomial'])(x)
            plt.plot(x, fit, label='Highest order term =' + str(i) + '\nR^2 = ' + str(poly['determination']))

        ref_max_key = max(ref_Rsq, key=lambda key: ref_Rsq[key])
        global ref_max_Rsq
        ref_max_Rsq = ref_Rsq[ref_max_key]
        plt.legend(loc='lower right')
        plt.xlabel('Wavelength[nm]')
        plt.ylabel('Measured transmissions[dB]')
        plt.title('Fitting Function')

        # Modeling
        plt.subplot(2, 2, 2)
        ref = np.poly1d(np.polyfit(x, y, ref_max_key))
        for wavelengthsweep in root.iter('WavelengthSweep'):
            L = list(map(float, wavelengthsweep.findtext('L').split(',')))
            IL = list(map(float, wavelengthsweep.findtext('IL').split(',')))
            l = np.array(L)
            il = np.array(IL)
            if IL == ref_IL:
                name = 'Reference'
            else:
                name = 'DCBias=' + str(wavelengthsweep.attrib['DCBias']) + 'V'
            plt.plot(l, il - ref(l), label=name)

        image_path = route.replace("\\", "/").split("/")
        sub_path = ''
        for i in range(-4, -1):
            sub_path += '/' + image_path[i]
        plt.legend(loc='lower right')
        plt.xlabel('Wavelength[nm]')
        plt.ylabel('Transmissions[dB]')
        plt.title('Transmission spectra - fitted')
        save_path = path.path() + '/result/graph/lot' + sub_path
        directory.create_folder(save_path)

        if show is True:
            plt.show()
        if save is True:
            plt.savefig(save_path + '/' + image_path[-1] + '.png')
        plt.close()

ImportError: attempted relative import with no known parent package

#### 4) src_path.py  
- We imported the os and defined the function path.
- The function checks the current working path divided by '\\\\' and returns it by putting '/' between the values.

In [9]:
import os


def path():
    p = '/'.join(os.getcwd().split("\\"))
    return p


#### 5) src_process.py  
- Through the 'glob' library, only certain files were printed using the path name of the file.
- We define the all function and visualize the state progress using 'tqdm'. 
- The definition of the wafer and coordinate functions is that flag = false is used to repeat the statement while the condition is false.

In [11]:
import glob
from glob import glob
from tqdm import tqdm
from . import path
from . import graph
from . import extract


def all(save, show):
    for data in tqdm(glob(path.path() + '/data/**/*LMZ*.xml', recursive=True)):
        graph.graph(data, save, show)
        extract.data_save(data)


def wafer(wafer, save, show):
    flag = False
    for data in tqdm(glob(path.path() + '/data/**/{}/**/*LMZ*.xml'.format(wafer), recursive=True)):
        graph.graph(data, save, show)
        extract.data_save(data)
        flag = True
    if flag is False:
        raise ValueError('Check Wafer Option')

ImportError: attempted relative import with no known parent package

#### 6) run.py
- We imported sys, a module that allows direct control of variables and functions provided by Python interpreters. 
- Dialog defined a new class in GUI programming, using a purpose-built dialog class as a window for user interaction. 
- We put wafer and coordinate as options to choose whether to show or save graphs.


In [13]:
import sys
from PyQt5.QtWidgets import *
from PyQt5.QtCore import *
from src import process


class CInputDialogWindow(QMainWindow):
    def __init__(self):
        super().__init__()
        self.setupUI()

    def setupUI(self):
        self.setWindowTitle("PyQtInputDialog 예제")
        self.setGeometry(100, 100, 200, 250)

        self.label = QLabel("Wafer : ", self)
        self.label.move(20, 20)
        self.label.resize(150, 20)

        self.label2 = QLabel("Column : ", self)
        self.label2.move(20, 20)
        self.label2.resize(150, 80)

        self.waferEdit = QLineEdit("All", self)
        self.waferEdit.move(80, 20)
        self.waferEdit.resize(100, 20)

        self.columnEdit = QLineEdit("All", self)
        self.columnEdit.move(80, 50)
        self.columnEdit.resize(100, 20)

        self.saveEdit = QCheckBox("Save", self)
        self.saveEdit.move(30, 80)
        self.saveEdit.toggle()

        self.showEdit = QCheckBox("Show", self)
        self.showEdit.move(30, 100)
        self.showEdit.toggle()

        self.btnSave = QPushButton("OK", self)
        self.btnSave.move(50, 200)
        self.btnSave.clicked.connect(self.btnInput_clicked)


    def btnInput_clicked(self):
        wafer = self.waferEdit.text()
        column = self.columnEdit.text()
        save = self.saveEdit.isChecked()
        show = self.showEdit.isChecked()
        try:
            if wafer == '' or column == '':
                raise ValueError('There is blank')
            if wafer == 'All' and column == 'All':
                process.all(save, show)
            elif wafer != 'All' and column == 'All':
                process.wafer(wafer, save, show)
            else:
                process.coordinate(wafer, column, save, show)
        except ValueError as e:
            print('Error: ', e, ', Pleas Input Again')
        #QCoreApplication.instance().quit()


if __name__ == "__main__":
    app = QApplication(sys.argv)
    window = CInputDialogWindow()
    window.show()
    app.exec_()

ModuleNotFoundError: No module named 'src'

#### 7) .gitignore
- We used gitignore to prevent loss due to data conflicts and to prevent unwanted files from being uploaded.

In [12]:
data/*

result/*

.idea/*

__pycache__/*

test.py

test2.py

src/__pycache__

SyntaxError: invalid syntax (<ipython-input-12-76736d79b648>, line 1)

# 4. Example of analysis result
### Show a sample image of analysis result  

![analysis result](https://cdn.discordapp.com/attachments/848924937712435253/848932787088261150/KakaoTalk_20210531_232448716_01.png)

### Show a table in csv file

![csv file](https://cdn.discordapp.com/attachments/848924937712435253/848932776405237770/KakaoTalk_20210531_232448716_02.png)